# Concat Operator

* https://xarray.pydata.org/en/stable/user-guide/combining.html
* https://xarray.pydata.org/en/stable/user-guide/io.html#reading-multi-file-datasets

In [1]:
import xarray as xr
import pandas as pd

In [2]:
from pathlib import Path
data_dir = Path("/Users/pingu/data/cmip6-decadal/orig")

In [3]:
cmip6_files = sorted(data_dir.glob('**/*.nc'))
cmip6_files

[PosixPath('/Users/pingu/data/cmip6-decadal/orig/tas_Amon_MPI-ESM1-2-HR_dcppA-hindcast_s1970-r10i1p1f1_gn_197011-198012.nc'),
 PosixPath('/Users/pingu/data/cmip6-decadal/orig/tas_Amon_MPI-ESM1-2-HR_dcppA-hindcast_s1970-r1i1p1f1_gn_197011-198012.nc'),
 PosixPath('/Users/pingu/data/cmip6-decadal/orig/tas_Amon_MPI-ESM1-2-HR_dcppA-hindcast_s1970-r2i1p1f1_gn_197011-198012.nc'),
 PosixPath('/Users/pingu/data/cmip6-decadal/orig/tas_Amon_MPI-ESM1-2-HR_dcppA-hindcast_s1970-r3i1p1f1_gn_197011-198012.nc'),
 PosixPath('/Users/pingu/data/cmip6-decadal/orig/tas_Amon_MPI-ESM1-2-HR_dcppA-hindcast_s1970-r4i1p1f1_gn_197011-198012.nc'),
 PosixPath('/Users/pingu/data/cmip6-decadal/orig/tas_Amon_MPI-ESM1-2-HR_dcppA-hindcast_s1970-r5i1p1f1_gn_197011-198012.nc'),
 PosixPath('/Users/pingu/data/cmip6-decadal/orig/tas_Amon_MPI-ESM1-2-HR_dcppA-hindcast_s1970-r6i1p1f1_gn_197011-198012.nc'),
 PosixPath('/Users/pingu/data/cmip6-decadal/orig/tas_Amon_MPI-ESM1-2-HR_dcppA-hindcast_s1970-r7i1p1f1_gn_197011-198012.nc'),

In [4]:
ds0 = xr.open_dataset(cmip6_files[0])
ds0

<xarray.Dataset>
Dimensions:    (time: 122, bnds: 2, lat: 192, lon: 384)
Coordinates:
  * time       (time) datetime64[ns] 1970-11-16 ... 1980-12-16T12:00:00
  * lat        (lat) float64 -89.28 -88.36 -87.42 -86.49 ... 87.42 88.36 89.28
  * lon        (lon) float64 0.0 0.9375 1.875 2.812 ... 356.2 357.2 358.1 359.1
    height     float64 ...
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) datetime64[ns] ...
    lat_bnds   (lat, bnds) float64 ...
    lon_bnds   (lon, bnds) float64 ...
    tas        (time, lat, lon) float32 ...
Attributes: (12/39)
    Conventions:           CF-1.7 CMIP-6.2
    activity_id:           DCPP
    branch_method:         standard lagged initialization
    contact:               cmip6-mpi-esm@dkrz.de
    creation_date:         2020-09-08T11:54:02Z
    data_specs_version:    01.00.30
    ...                    ...
    title:                 MPI-ESM1-2-HR output prepared for CMIP6
    variable_id:           tas
    variant_label:         r10i1p1f1
    license:               CMIP6 model data produced by MPI-M is licensed und...
    cmor_version:          3.5.0
    tracking_id:           hdl:21.14100/f7b31263-2d25-4b36-b8b3-1eacfd30750d

In [5]:
ds0.realization_index

10

In [6]:
datasets = [xr.open_dataset(f) for f in cmip6_files]
len(datasets)

10

In [7]:
realization_values = [ds.realization_index for ds in datasets]
realization_values

[10, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [8]:
realization_dim = pd.Index(realization_values, name="realization")

In [9]:
ds_concat = xr.concat(datasets, dim=realization_dim)
ds_concat

<xarray.Dataset>
Dimensions:      (time: 122, realization: 10, bnds: 2, lat: 192, lon: 384)
Coordinates:
  * time         (time) datetime64[ns] 1970-11-16 ... 1980-12-16T12:00:00
  * lat          (lat) float64 -89.28 -88.36 -87.42 -86.49 ... 87.42 88.36 89.28
  * lon          (lon) float64 0.0 0.9375 1.875 2.812 ... 357.2 358.1 359.1
    height       float64 2.0
  * realization  (realization) int64 10 1 2 3 4 5 6 7 8 9
Dimensions without coordinates: bnds
Data variables:
    time_bnds    (realization, time, bnds) datetime64[ns] 1970-11-01 ... 1981...
    lat_bnds     (realization, lat, bnds) float64 -89.75 -88.82 ... 88.82 89.75
    lon_bnds     (realization, lon, bnds) float64 -0.4688 0.4688 ... 358.6 359.5
    tas          (realization, time, lat, lon) float32 232.8 232.8 ... 245.2
Attributes: (12/39)
    Conventions:           CF-1.7 CMIP-6.2
    activity_id:           DCPP
    branch_method:         standard lagged initialization
    contact:               cmip6-mpi-esm@dkrz.de
    creation_date:         2020-09-08T11:54:02Z
    data_specs_version:    01.00.30
    ...                    ...
    title:                 MPI-ESM1-2-HR output prepared for CMIP6
    variable_id:           tas
    variant_label:         r10i1p1f1
    license:               CMIP6 model data produced by MPI-M is licensed und...
    cmor_version:          3.5.0
    tracking_id:           hdl:21.14100/f7b31263-2d25-4b36-b8b3-1eacfd30750d

In [10]:
ds_concat.isel(realization=0)

<xarray.Dataset>
Dimensions:      (time: 122, bnds: 2, lat: 192, lon: 384)
Coordinates:
  * time         (time) datetime64[ns] 1970-11-16 ... 1980-12-16T12:00:00
  * lat          (lat) float64 -89.28 -88.36 -87.42 -86.49 ... 87.42 88.36 89.28
  * lon          (lon) float64 0.0 0.9375 1.875 2.812 ... 357.2 358.1 359.1
    height       float64 2.0
    realization  int64 10
Dimensions without coordinates: bnds
Data variables:
    time_bnds    (time, bnds) datetime64[ns] 1970-11-01 ... 1981-01-01
    lat_bnds     (lat, bnds) float64 -89.75 -88.82 -88.82 ... 88.82 88.82 89.75
    lon_bnds     (lon, bnds) float64 -0.4688 0.4688 0.4688 ... 358.6 358.6 359.5
    tas          (time, lat, lon) float32 232.8 232.8 232.9 ... 245.2 245.2
Attributes: (12/39)
    Conventions:           CF-1.7 CMIP-6.2
    activity_id:           DCPP
    branch_method:         standard lagged initialization
    contact:               cmip6-mpi-esm@dkrz.de
    creation_date:         2020-09-08T11:54:02Z
    data_specs_version:    01.00.30
    ...                    ...
    title:                 MPI-ESM1-2-HR output prepared for CMIP6
    variable_id:           tas
    variant_label:         r10i1p1f1
    license:               CMIP6 model data produced by MPI-M is licensed und...
    cmor_version:          3.5.0
    tracking_id:           hdl:21.14100/f7b31263-2d25-4b36-b8b3-1eacfd30750d

## average over dim realization

In [11]:
ds_mean = ds_concat.mean(dim=realization_dim.name, skipna=True, keep_attrs=True)
ds_mean

<xarray.Dataset>
Dimensions:   (time: 122, lat: 192, bnds: 2, lon: 384)
Coordinates:
  * time      (time) datetime64[ns] 1970-11-16 ... 1980-12-16T12:00:00
  * lat       (lat) float64 -89.28 -88.36 -87.42 -86.49 ... 87.42 88.36 89.28
  * lon       (lon) float64 0.0 0.9375 1.875 2.812 ... 356.2 357.2 358.1 359.1
    height    float64 2.0
Dimensions without coordinates: bnds
Data variables:
    lat_bnds  (lat, bnds) float64 -89.75 -88.82 -88.82 ... 88.82 88.82 89.75
    lon_bnds  (lon, bnds) float64 -0.4688 0.4688 0.4688 ... 358.6 358.6 359.5
    tas       (time, lat, lon) float32 231.8 231.8 231.8 ... 248.2 248.2 248.2
Attributes: (12/39)
    Conventions:           CF-1.7 CMIP-6.2
    activity_id:           DCPP
    branch_method:         standard lagged initialization
    contact:               cmip6-mpi-esm@dkrz.de
    creation_date:         2020-09-08T11:54:02Z
    data_specs_version:    01.00.30
    ...                    ...
    title:                 MPI-ESM1-2-HR output prepared for CMIP6
    variable_id:           tas
    variant_label:         r10i1p1f1
    license:               CMIP6 model data produced by MPI-M is licensed und...
    cmor_version:          3.5.0
    tracking_id:           hdl:21.14100/f7b31263-2d25-4b36-b8b3-1eacfd30750d

In [12]:
ds_mean.to_netcdf("out/cmip6_decdal_mean.nc")